<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **COVID Dashboard**

## 1\. Contexto

Este será um projeto em parceria com a ebac aonde temos dados da covid-19.

Iremos extrai-los, trata-los e o salvar corretamente para que possamos estar fazendo um dashboard com algumas informações da covid-19 no Brasil no ano de 2021.

Teremos dois modelo de dados, um sobre os casos registrados e um sobre a vacinação.

Este será o de casos registrados:

## 2\. Pacotes e bibliotecas

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

## 3\. Extração

In [ ]:
# Casos:
# Vamos processar os dados de casos da universidade John Hopkins.
# • Extração: O dado está compilado em um arquivo por dia, exemplo para 2021/12/01.

cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',' )

# Portanto, precisaremos iterar dentro de um intervalo de tempo definido para extraí-lo:

def date_range(start_date:datetime, end_date:datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

start_date = datetime(2021,1,1)
end_date = datetime(2021, 12, 31)

# De maneira iterativa, vamos selecionar as colunas de interesse e as linhas referentes ao Brasil:

cases = None
cases_is_empty = True
for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop([ 'FIPS', 'Admin2', 'Last_Update' , 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key' , 'Case_Fatality_Ratio'] , axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = cases.append(case, ignore_index=True)


# Começamos com o nome das colunas:

for col in cases.columns:
  cases = cases.rename(columns={col:col.lower()})

cases = cases.rename(columns={'province_state':'state', 'country_region':'country'})

# Ajustamos o nome dos estados:

states_map = {'Amapa':'Amapá', 'Ceara':'Ceará', 'Espirito Santo':'Espírito Santo',
              'Goias':'Goiás', 'Para':'Pará', 'Paraiba':'Paraíba', 'Parana':'Paraná', 'Piaui':'Piauí', 'Rondonia':'Rondônia', 'Sao Paulo':'São Paulo' }

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

## 4\. Transformação

In [ ]:
# Vamos, então, computar novas colunas para enriquecer a base de dados.
# - Chaves temporais:

cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year'] = cases['date'].apply(lambda date: date.strftime('%Y'))

# - População estimada do estado:

cases['population'] = round( 100000 * (cases['confirmed'] / cases['incident_rate']) )
cases = cases.drop('incident_rate', axis=1)

# - Número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado:

cases_ = None
cases_is_empty = True

def get_trend(rate:float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.85: # No material de apoio esta como 0.75, porem pelo fato da variação ser de 0.15 +-, o certo acaba sendo 0.85.
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status

for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values( by=['date'] )

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff( periods=1 )
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil( cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d'] / cases_per_state['confirmed_moving_avg_7d'].shift(periods=14 )
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff( periods=1 )
  cases_per_state['deaths_moving_avg_7d'] = np.ceil( cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d'] / cases_per_state['deaths_moving_avg_7d'].shift(periods=14 )
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = cases_.append(cases_per_state, ignore_index=True)

cases = cases_
cases_ = None

# Vamos fazer o type casting das colunas:

cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

# Por fim, vamos reorganizar as colunas.

cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d',
               'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d',
               'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year' ]]

## 5\. Carregamento

In [ ]:
# • Carregamento: Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carregá-lo no Google Data Studio:

cases.to_csv('./covid-cases.csv', sep=',', index=False)

## 6. Link do notebook e do google data studio:
No google colab com ambos os dados, tanto casos quanto vacinações:  
-https://colab.research.google.com/drive/1uNk3G9-ZpyDsZ-PP6xdUBu6nwzJywF4F?usp=sharing               

já no google data studio contém o dashboard dos dados:  
-https://lookerstudio.google.com/reporting/a4b9b68e-83df-42e6-8b36-a85a65999de2